## Week 6:  Loading and Cleaning Data

Today's lecture will cover the basics of loading data into R and cleaning it. In class and online tutorials (like DataCamp) the datasets you're given are impeccably clean and ready to go immediately. In practice this is simply not the case--75% of a data scientist's time is used for getting and cleaning data prior to modeling! So today you will learn the basics of loading common static data formats and cleaning the data using [regular expressions]().

Up until now the instructors have been writing the code for reading and loading data to R for you. From this tutorial forward we will leave the loading to you.

## Common Data Formats

<strong><a href="https://en.wikipedia.org/wiki/Comma-separated_values">CSV</a></strong> stands for Comma-Seperated Values. These are text files, where each line is an observation and the variables are seperated by commas. In a <code>.csv</code> file, the <strong>delimiter</strong> is the comma. There are similar offshoots of this format: tab-seperated values or a general delimiter-seperated values. If a <code>.csv</code> file includes the header, it is entered as the first line in the file. Also notice in the sample that there is a comma included in the <code>name</code> field--how is that possible?

    "Date","Name","Grade"
    "25 May","Bloggs, Fred","C"
    "25 May","Doe, Jane","B"
    "15 July","Bloggs, Fred","A"
    "15 April","Muniz, Alvin ""Hank""","A"
    
<strong><a href="http://json.org/">JSON</a></strong> stands for Javascipt Object Notation. This data storage format is very common in application and website databases because it is lightweight and flexible. The example below holds two records, one for John Smith and another for Jason Freeberg. As you can see, the two records have different numbers of fields--the first does not have a <code>middlename</code> and the second does not have <code>postalCode2</code>. If we attempted to store this in a tabular format, we would have many missing entries and waste space.

    {
      "firstName": "John",
      "lastName": "Smith",
      "isAlive": true,
      "age": 25,
      "address": {
        "streetAddress": "21 2nd Street",
        "city": "New York",
        "state": "NY",
        "postalCode1": 3100
        "postalCode2": 10021
      }
    },
    {
      "firstName": "Jason",
      "middleName": "Robert",
      "lastName": "Freeberg",
      "isAlive": true,
      "age": 21,
      "address": {
        "streetAddress": "6760 Sabado",
        "streetAddress2": "Unit B"
        "city": "Isla Vista",
        "state": "CA",
        "postalCode1": 93117
      }
    }

## Loading Data in R

Since R is a statistical software, and statistics is the analysis of data, it makes sense that R has many functions built in for loading data. R's general <code>read.table()</code> function will read tabular data but requires that you specify the file's delimiter.

In [1]:
# Package for working with JSON
#install.packages("rjson")
library(rjson)

# Let's load last week's data as a refresher
arrests <- read.table("nfl_arrests.csv", header=T, sep=",")
head(arrests)

# Now let's try a JSON collection from the internet
jsonFile <- "http://api.worldbank.org/country?per_page=10&region=OED&lendingtype=LNX&format=json"
jsonDoc <- fromJSON(paste(readLines(jsonFile), collapse=""))
str(jsonDoc)
firstRecord


season,week_num,day_of_week,gametime_local,home_team,away_team,home_score,away_score,OT_flag,arrests,division_game
2011,1,Sunday,1:15:00 PM,Arizona,Carolina,28,21,,5,n
2011,4,Sunday,1:05:00 PM,Arizona,New York Giants,27,31,,6,n
2011,7,Sunday,1:05:00 PM,Arizona,Pittsburgh,20,32,,9,n
2011,9,Sunday,2:15:00 PM,Arizona,St. Louis,19,13,OT,6,y
2011,13,Sunday,2:15:00 PM,Arizona,Dallas,19,13,OT,3,n
2011,14,Sunday,2:05:00 PM,Arizona,San Francisco,21,19,,4,y


Warning message in readLines(jsonFile):
“incomplete final line found on 'http://api.worldbank.org/country?per_page=10&region=OED&lendingtype=LNX&format=json'”

List of 2
 $ :List of 4
  ..$ page    : num 1
  ..$ pages   : num 4
  ..$ per_page: chr "10"
  ..$ total   : num 31
 $ :List of 10
  ..$ :List of 10
  .. ..$ id         : chr "AUS"
  .. ..$ iso2Code   : chr "AU"
  .. ..$ name       : chr "Australia"
  .. ..$ region     :List of 2
  .. .. ..$ id   : chr "EAS"
  .. .. ..$ value: chr "East Asia & Pacific"
  .. ..$ adminregion:List of 2
  .. .. ..$ id   : chr ""
  .. .. ..$ value: chr ""
  .. ..$ incomeLevel:List of 2
  .. .. ..$ id   : chr "HIC"
  .. .. ..$ value: chr "High income"
  .. ..$ lendingType:List of 2
  .. .. ..$ id   : chr "LNX"
  .. .. ..$ value: chr "Not classified"
  .. ..$ capitalCity: chr "Canberra"
  .. ..$ longitude  : chr "149.129"
  .. ..$ latitude   : chr "-35.282"
  ..$ :List of 10
  .. ..$ id         : chr "AUT"
  .. ..$ iso2Code   : chr "AT"
  .. ..$ name       : chr "Austria"
  .. ..$ region     :List of 2
  .. .. ..$ id   : chr "ECS"
  .. .. ..$ value: chr "Europe & Central Asia"
  .. ..$ adminregion:List of 2
 

ERROR: Error in eval(expr, envir, enclos): object 'firstRecord' not found


In [2]:
# Use read.table() to import "arrests.txt"

#dirtyData <- read.table(<FILL-IN>)
dirtyData <- read.table("arrests.txt", header=T, sep="\t")
head(dirtyData)

# Hint: open the file in a text editor--how is this file different from a .csv?

,season,week_num,day_of_week,gametime_local,home_team,away_team,home_score,away_score,OT_flag,arrests,division_game
689,2011,4,Sunday,1:00:00 PM,Philadelphia,San Francisco,23,24,,5,n
745,2013,3,Sundayoops!,8:30:00 PM,Pittsburgh,123Chicago,23,40,,56,n
964,2015,12,Sunday,12:00:00 PM,Tennessee,Oakland,21,24,,0,n
448,2014,5,Sunday,1:00:00 PM,Jacksonville,Pittsburgh,9,17,,3,n
666,2012,15,Sunday,1:25:00 PM,Oakland,Kansas City,0,15,,8,y
21,2013,10,Sundayoops!,2:25:00 PM,Arizona,Houston,27,24,,3,n


## Checking Data

In data analysis, there is a phase that precedes modeling and we call it <strong>exploratory analysis</strong>. This step involves familiarizing yourself with the data by checking the dimensions, understanding the variables, making visualizations and performing other sanity checks. Our last topic of the quarter will be data visualization with ggplot2, so let's cover the other ways we can explore our data.

The list below are some tips that I have found very useful when I'm doing my exploratory analysis.
<ul>
    <li>
    Use <code>dim()</code> to get the dimensions of the dataframe. The functions <code>summary()</code> and <code>glimpse()</code> are great for orienting yourself with a new dataset.
    </li>
    <li>
    Always understand the units and range of your numeric variables. Similarly, understand the naming convention of factor levels within categorical variables.
    </li>
    <li>
    Use <code>max()</code> and <code>min()</code> to check for odd values in numeric variables. And use <code>unique()</code> to check for incorrect levls in categorical variables.
    </li>
    <li>
    <code>table()</code> is great for getting frequency counts of the factors within categorical variables. You can give it two categorical variables to get two-way tables as well.
    </li>
</ul>

In [4]:
# Get the dimensions of dirtyData
#<FILL-IN>

# Try using glimpse() (from the dplyr library) on dirtyData
library(plyr)
library(dplyr)
#<FILL-IN>

# Get the summary of the arrests column
#<FILL-IN>

# Call unique() on the weekday, away team, and division game columns
uniqueWeek <- unique(dirtyData$day_of_week)
uniqueTeam <- unique(dirtyData$away_team)
uniqueDiv <- unique(dirtyData$division_game)

#uniqueWeek <- <FILL-IN>
#uniqueTeam <- <FILL-IN>
#uniqueDiv <- <FILL-IN>

uniqueWeek
uniqueTeam
uniqueDiv


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



[1] Sunday        Sundayoops!   Mondayoops!   Thursdayoops! Thursday     
[6] Monday        Saturday     
7 Levels: Monday Mondayoops! Saturday Sunday Sundayoops! ... Thursdayoops!

[1] San Francisco      123Chicago         Oakland            Pittsburgh        
 [5] Kansas City        Houston            Baltimore          San Diego         
 [9] Green Bay          New York Jets      Tennessee          Buffalo           
[13] Carolina           Philadelphia       Miami              123Jacksonville   
[17] Cincinnati         123New England     Arizona            St. Louis         
[21] 123Tennessee       123Minnesota       New York Giants    123Oakland        
[25] New Orleans        123Cleveland       Denver             123Green Bay      
[29] Chicago            Detroit            123Philadelphia    Seattle           
[33] 123Dallas          Jacksonville       123Tampa Bay       123Denver         
[37] 123Seattle         123Atlanta         Cleveland          Minnesota         
[41] Indianapolis       123Houston         123Baltimore       123Detroit        
[45] Washington         New England        Atlanta            Dallas            
[49] 123Indianapolis    123Arizona         123Miami           Tampa Bay         
[53] 123San Francisco   123New Orleans     123Carolina        123New York Jets  
[57] 123Washington      123Cincinnati      123New York Giants 123Pittsburgh     
60 Levels: 123Arizona 123Atlanta 123Baltimore 123Carolina ... Washington

[1] n y
Levels: n y

## Regular Expressions

<em>Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems.</em>

Regular expressions define a syntax of characters that can be used to match patterns in strings. Moreover, they can be used by search algorithms to find matches, or even replace the matches with other characters. If you're observant, you noticed that the data above has some odd values--like "oops!" in <code>day_of_week</code> and 123 in <code>away_team</code>. We can use R's functions <code>grep()</code> and <code>grepl()</code> to find matches, or <code>gsub()</code> to replace matches with other characters.

### Common Expressions and Characters

We will only scatch the surface of regular expressions here. Last week's homework was to work through <a href="https://regexone.com">regexone's tutorials</a>. If you didn't do that assignment, find a student that did to assist you. Let's take a look at some example regular expressions before diving into the syntax.

<table>
    <tr>
        <td>
        Expression
        </td>
        <td>
        Matches
        </td>
        <td>
        Example Matches (in <strong>bold</strong>)
        </td>
        <td>
        Does not match
        </td>
    </tr>
    <tr>
        <td>
            "abc"
        </td>
        <td>
            "abc" literally
        </td>
        <td>
            <strong>abc</strong>, z<strong>abc</strong>, <strong>abc</strong>red
        </td>
        <td>
            acb
        </td>
    </tr>
    <tr>
        <td>
            "\d\d\d"
        </td>
        <td>
            Sequence of any 3 uninterrupted digits.
        </td>
        <td>
            <strong>123</strong>, <strong>805</strong>, <strong>415</strong>345, wow<strong>302</strong>such
        </td>
        <td>
            word, wow, noNumbersHere!
        </td>
    </tr>
    <tr>
        <td>
            "[a-zA-Z]"
        </td>
        <td>
            Single letter, upper or lower case
        </td>
        <td>
            2<strong>b</strong>4, <strong>w</strong>ww.wow.com, #&&2&3<strong>A</strong>
        </td>
        <td>
            234, @@@@, 123\_\_405
        </td>
    </tr>
    <tr>
        <td>
            "[a-zA-Z]+"
        </td>
        <td>
            Sequence of letters, upper or lower case
        </td>
        <td>
            0<strong>qwerty</strong>0, <strong>www</strong>.wow.com, #&&2&3<strong>Az</strong>2aa
        </td>
        <td>
            234, @@@@, 123\_\_405
        </td>
    </tr>
    <tr>
        <td>
            "[^0-9]+"
        </td>
        <td>
            Sequence of characters not containing a digit
        </td>
        <td>
            <strong>I'm at the </strong>805, <strong>onlyLetters</strong>, <strong>st</strong>0p, 0000<strong>onlyThis,Man!</strong>
        </td>
        <td>
            123, 4158051122
        </td>
    </tr>
</table>

Let's cover some important characters in regular expressions.

<ul>
    <li>
        <strong>Metacharacters</strong> are characters that have special meaning. In regular expessions, some metacharacters are <code>[</code>, <code>]</code>, <code>^</code>, <code>.</code>, <code>|</code>, and <code>+</code>... just to name a few. These characters will not match their literal meaning (the substring, "[ ]", will <strong>not</strong> find "[ ]" in the string) and must be escaped with a "\" if you want them to be literal.
        <ul>
            <li>
                To find the substring <code>"(1+1)\*3=6"</code> in a string, your regular expression would be <code>"\\(1\\+1\\)\\*3=6"</code> because "(", ")", and "+" have special meaning when they are not escaped.
            </li>
        </ul>
    </li>
    <li>
        <strong>Square Brackets</strong> will match their contents <strong>once</strong>. For example, "<code>[jump]</code>" will match a <strong>single</strong> character of "j", "u", "m", or "p". Not the string, "jump". But we can make some modifications...
        <ul>
            <li>
                A caret, ^, inside the square brackets will make the expression match a single character <strong>not</strong> in the list. "<code>[^jump]</code>" will match any one character that is <strong>not</strong> "j", "u", "m", or "p".
            </li>
            <li>
                An addition symbol, +, just outside the square brackets will make the expression match the bracketed list any number of times! "<code>[jump]+</code>" will match "jump", "jjuummpp", and "ppmmjjuujj".
            </li>
        </ul>
    </li>
    <li>
        <strong>The Wildcard</strong> is the period. It will match <strong>any</strong> character, and should be used with care. "<code>...</code>" will match "red", "_!^" and "<strong>blu</strong>e".
    </li>
    <li>
        <strong>The logical operator for "<code>or</code>" ("<code>|</code>")</strong>, still works within regular expressions. The expression, "this|that" will either match the literals "this" or "that" once in a string. 
    </li>
</ul>


### Regex Tips

<ul>
    <li>
    <strong>Do not make one big regular expression.</strong> Break down the regex into smaller, <em>more manageable</em>, problems. Use comments to help yourself keep track of the expressions.
    </li>
    <li>
    <strong>Use [Regex101.com](https://regex101.com).</strong> This website will check your expression against sample text. The top right breaks down your regex character-by-character, letting you know what it is <em>and is not</em> capturing. In the bottom right, there is a small window with common tokens and expressions.
    </li>
    <li>
    <strong>Test your regex.</strong> Double-check the expression worked by using it on examples where you know the expected outcome. 
    </li>
</ul>

### R's family of grep functions

<strong>g/re/p</strong>: <em><strong>g</strong>lobally search a <strong>re</strong>gular expression and <strong>p</strong>rint.</em>

R has a great collection of functions for regular expression operations right in its base library.

<ul>
    <li>
        <code>grep(<em>pattern, x, ignore.case, ...</em>)</code> searches for matches of the <code>pattern</code> in <code>x</code>. Similar functions like <code>regexpr(), gregexpr() and regexec()</code> provide the same general utility but differ in the amount of information returned. <code>grepl()</code> returns a logical.
    </li>
    <li>
        <code>gsub(<em>pattern, replacement, x, ignore.case</em>)</code> will <strong>sub</strong>stitute any matches of <code>pattern</code> with the <code>replacement</code>. This is the function we will use to <em>clean</em> data, because we can remove any matches by simply replacing it with the empty string, "".
    </li>
</ul>

In [18]:
# Here's a vector that should only be numeric,
# but there is some contamination.
dirtyVector <- c("1", "2", "3q", "4r", "5we", "6butt", "7WOW")

# So we need to make a regex to match the letters
r1 <- "[a-zA-Z]"

# Pass r1 as the pattern, and the replacement is the empty string
cleanedVector <- gsub(pattern=r1, replacement="", x=dirtyVector)

# Notice that I coerced the vector to a numeric without any errors
print(paste("After cleaning...", paste(as.numeric(cleanedVector), collapse=",")))

# Now an even dirtier vector that should be numeric
dirtyVector <- c("1..", "qq2", "3ere", "4ee", "(5)", "6'er", "~7~")

# Same process... what is r2 matching?
r2 <- "[^0-9]+"
cleanedVector <- gsub(pattern=r2, replacement="", x=dirtyVector)
print(paste("After cleaning...", paste(as.numeric(cleanedVector), collapse=",")))

[1] "After cleaning... 1,2,3,4,5,6,7"
[1] "After cleaning... 1,2,3,4,5,6,7"


## Exercises

Loading static data is a pretty trivial process, so the exercises will focus on cleaning data with regular expressions.

## Congratulations!

You're done with tonight's exercises! Check [the syllabus](https://github.com/JasonFreeberg/R_Tutorials/blob/master/README.md) for this week's homework. And don't forget... *if you're going through hell, you keep going.*